In [1]:
import os
import glob

import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import f_oneway
import scipy.stats as stats

In [2]:
measurements1 = '/home/konsternacja/Git/master-thesis1/fluorCAM/measurement1'
measurements2 = '/home/konsternacja/Git/master-thesis1/fluorCAM/measurement2'
measurements3 = '/home/konsternacja/Git/master-thesis1/fluorCAM/measurement3'

In [3]:
def flatten_sum(matrix):
    return sum(matrix, [])

In [4]:
def combine_reports(directory_path):
    
    files = glob.glob(directory_path + '*.TXT')

    # Initialize an empty list to store DataFrames
    dfs = []
    group_list = []

    for file in files:
        # Get the base name of the file without extension
        base_name = os.path.splitext(os.path.basename(file))[0]
    
        # Read the file into a DataFrame
        df = pd.read_csv(file, sep='\t', skiprows=2, index_col=0)
    
        # Append the DataFrame to the list
        dfs.append(df)
    
        # Add an entry to the group dictionary
        if 'wteminus' in base_name:
            group_list.append([0] * len(df.columns))
        elif 'wteplus' in base_name:
            group_list.append([1] * len(df.columns))
        elif '70eminus' in base_name:
            group_list.append([2] * len(df.columns))
        elif '70eplus' in base_name:
            group_list.append([3] * len(df.columns))
        else:
            group_list.append([None] * len(df.columns))
            
    # Concatenate the DataFrames along the common column (Unnamed: 0)
    result_df = pd.concat(dfs, ignore_index=True, axis=1)

    group_list_flat = flatten_sum(group_list)

    # Convert the group_dict to a DataFrame and transpose it
    group_df_flat = pd.DataFrame(group_list_flat)

    # Transpose group_df_flat so it can be appended as a row
    group_df_flat = group_df_flat.transpose()

    concatenated_df = pd.concat([result_df, group_df_flat], ignore_index=True)
    
    # Read the file
    file = pd.read_csv(directory_path + '70eminus1.TXT', sep='\t', skiprows=2)

    # Extract the index column
    index_column = file['Unnamed: 0']

    # Insert the index_column 
    df_for_analysis = concatenated_df.join(index_column)

    df_for_analysis = df_for_analysis.T

    # Set the last row as the header
    df_for_analysis.columns = df_for_analysis.iloc[-1]

    # Remove the last row
    df_for_analysis = df_for_analysis.iloc[:-1]

    # Rename the last column
    df_for_analysis = df_for_analysis.rename(columns={df_for_analysis.columns[-1]: 'groups'})

    # Save the concatenated DataFrame to a new TSV file
    df_for_analysis.to_csv(directory_path + 'concatenated_file.tsv', sep='\t', index=False)
    return df_for_analysis

In [5]:
df1 = combine_reports('/home/konsternacja/Git/master-thesis1/fluorCAM/measurement1/')
df2 = combine_reports('/home/konsternacja/Git/master-thesis1/fluorCAM/measurement2/')
df3 = combine_reports('/home/konsternacja/Git/master-thesis1/fluorCAM/measurement3/')

In [6]:
def plot_standardized(df):
    
    normalized_df = df.apply(lambda x: (x - x.min()) / (x.max() - x.min()) if x.name != 'groups' and x.name != 'Unnamed: 0' else x)

    # Grupowanie danych według kolumny 'groups'
    grouped_data = normalized_df.groupby('groups')
    grouped_data.head()

    # Przeprowadzenie testu jednoczynnikowego ANOVA dla każdej kolumny
    significant_columns = []
    for column in normalized_df.columns[1:-1]:  # Wykluczamy ostatnią kolumnę ('groups')
        group_values = [group[column].values for name, group in grouped_data]
        stat, p_value = f_oneway(*group_values)
        
        # Analiza wyników testu
        if p_value < 0.05:  # Próg istotności, można dostosować
            print(f'Różnice statystyczne w kolumnie "{column}" (p-value: {p_value})')
            significant_columns.append(column)
        # else:
        #     print(f'Brak różnic statystycznych w kolumnie "{column}" (p-value: {p_value})')

    # Możesz również przeprowadzić testy dla całej tabeli
    all_data_values = [group[column].values for name, group in grouped_data for column in normalized_df.columns[:-1]]
    stat, p_value = f_oneway(*all_data_values)

    # Analiza wyników testu dla całej tabeli
    if p_value < 0.05:  # Próg istotności, można dostosować
        print(f'Różnice statystyczne w całej tabeli (p-value: {p_value})')
    else:
        print(f'Brak różnic statystycznych w całej tabeli (p-value: {p_value})')
        
    # for col in significant_columns:
    #     plt.figure(figsize=(8, 6))
    #     normalized_df.boxplot(column=column, by='groups', grid=False)
    #     plt.title(f'Boxplot dla kolumny "{column}"')
    #     plt.suptitle('')  # Usunięcie domyślnego tytułu nad wykresem
    #     plt.show()

In [7]:
plot_standardized(df1)
plot_standardized(df2)
plot_standardized(df3)

Różnice statystyczne w kolumnie "Fm_L2" (p-value: 0.00423288065130578)
Różnice statystyczne w kolumnie "Fm_L3" (p-value: 0.00650417171240191)
Różnice statystyczne w kolumnie "Fm_L4" (p-value: 0.008125128695912564)
Różnice statystyczne w kolumnie "Fm_Lss" (p-value: 0.00728516838879779)
Różnice statystyczne w kolumnie "Ft_L2" (p-value: 0.0026128995312019957)
Różnice statystyczne w kolumnie "Ft_L3" (p-value: 0.0035435800754261733)
Różnice statystyczne w kolumnie "Ft_L4" (p-value: 0.009160332790443461)
Różnice statystyczne w kolumnie "Ft_Lss" (p-value: 0.00863332439775719)
Różnice statystyczne w kolumnie "Fv_L2" (p-value: 0.000595619756210967)
Różnice statystyczne w kolumnie "Fv_L3" (p-value: 0.0011731736119834477)
Różnice statystyczne w kolumnie "Fv_L4" (p-value: 0.0013756072481112256)
Różnice statystyczne w kolumnie "Fv_Lss" (p-value: 0.0010870372664053734)
Różnice statystyczne w kolumnie "Fq_L1" (p-value: 0.008061392369368604)
Różnice statystyczne w kolumnie "Fq_L2" (p-value: 0.01437053

In [8]:
def plot(signif_cols, normalized_df):
    for col in signif_cols:
        plt.figure(figsize=(8, 6))
        normalized_df.boxplot(column=column, by='groups', grid=False)
        plt.title(f'Boxplot dla kolumny "{column}"')
        plt.suptitle('')  # Usunięcie domyślnego tytułu nad wykresem
        plt.show()

In [9]:
plot(signif_cols1, df1)

NameError: name 'signif_cols1' is not defined